In [1]:
import networkx
import numpy as np

In [2]:
c=[[0 for x in range(21)] for x in range(21)]

In [3]:
def connect(c,a,b):
    c[a][b]=c[b][a]=1

In [4]:
connect(c,0,9)
connect(c,1,18)
connect(c,2,13)
connect(c,1,7)
connect(c,4,10)
connect(c,4,20)
connect(c,7,8)
connect(c,9,16)
connect(c,11,17)
connect(c,17,18)
connect(c,18,19)
connect(c,12,18)
connect(c,6,12)
connect(c,9,10)
connect(c,18,13)
connect(c,14,15)
connect(c,19,13)
print(c[7][1],c[1][7])

1 1


In [5]:
active=np.array([0,1,2,4,9,13,17,18,19])
con=np.array(c)
red=con[active][:,active]
for i in range(len(red)):
    for j in range(len(red)):
        if red[i,j]!=0:
            print(i,j,red[i,j])

0 4 1
1 7 1
2 5 1
4 0 1
5 2 1
5 7 1
5 8 1
6 7 1
7 1 1
7 5 1
7 6 1
7 8 1
8 5 1
8 7 1


In [7]:
def get_new_neighbours(red, index, l):
    neigh=np.arange(0,len(red),1)[red[index].astype(np.bool8)]
    mask=np.isin(neigh, l, invert=True)
    return neigh[mask]

In [39]:
def get_families(red):
    n_active=len(red)
    n_fam=-1
    l=[]
    ll=[] #families
    
    def save(neigh, fam, ll, l):
        ll[fam]=ll[fam]+ neigh.tolist()
        l=l+neigh.tolist()
        print("saving")
        print("--> ll ", ll)
        print("--> l ", l)
        return ll,l
    
    def get_active_connected(red,i, ll, l, n_fam):
        new_neigh=get_new_neighbours(red, i, l)
        print("got neighbours of node",i)
        print("--> new_neigh:",new_neigh)
        if new_neigh.size!=0:
            print("not an empty list, now saving")
            ll,l=save(new_neigh, n_fam, ll, l)
            print("begin loop on ", new_neigh)
            for j in new_neigh:
                print("get all active connections of ", j)
                ll,l=get_active_connected(red, j, ll, l, n_fam)
        else:
            print("empty list")
        return ll, l
    
    for i in range(n_active):
        print("trying ", i)
        if i not in l:
            print("begin new family")
            print("get all active connections of ", i)
            n_fam+=1
            ll.append([])
            ll[n_fam].append(i)
            l.append(i)
            ll,l=get_active_connected(red,i, ll, l, n_fam) 
        else:
            print("already in l")
    return ll,l

## Mixed python code

In [98]:
def get_families(red):
    n_active=len(red)
    n_fam=-1
    l=[]
    ll=[] #families
    
    def save(neigh, fam, ll, l):
        ll[fam]=ll[fam]+ neigh.tolist()
        l=l+neigh.tolist()
        return ll,l
    
    def get_active_connected(red,i, ll, l, n_fam):
        new_neigh=get_new_neighbours(red, i, l)
        if new_neigh.size!=0:
            ll,l=save(new_neigh, n_fam, ll, l)
            for j in new_neigh:
                ll,l=get_active_connected(red, j, ll, l, n_fam)
        return ll, l
    
    for i in range(n_active):
        if i not in l:
            n_fam+=1
            ll.append([])
            ll[n_fam].append(i)
            l.append(i)
            ll,l=get_active_connected(red,i, ll, l, n_fam) 
    return sorted(list(map(len ,ll)), reverse=True)

In [102]:
%time
get_families(red)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10.7 µs


[6, 2, 1]